In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
data_folder="drive//MyDrive//Colab Notebooks//dataset" # dataset folder

In [4]:
folders=['0','1','2','3','4','5','6'] #folders in dataset folder
'''folders=['0','3','4','5'] #reduced emotions
folders=['5'] #reduced emotions
folders=['0','3','5'] #folders in dataset folder'''


"folders=['0','3','4','5'] #reduced emotions\nfolders=['5'] #reduced emotions\nfolders=['0','3','5'] #folders in dataset folder"

In [6]:
# Read all images and convert them into array

In [9]:
imgSize=224 #we will change size because mobileNet accepts 224*224 in rgb
training_data=[] # a data list
def create_training_data():

    for folder in folders:
        path= os.path.join(data_folder,folder)
        print(os.path.join(data_folder,folder))
        #id=int(folder)
        id= folders.index(folder) # get id for each image, id is index of each emotion folder
        for img in os.listdir(path):
            try:
                imgArray= cv2.imread(os.path.join(path,img))
                print(os.path.join(path,img))
                resizeArray= cv2.resize(imgArray,(imgSize,imgSize)) # resizing the images from 48*48 ti 224*224
                training_data.append([resizeArray,id]) # push image and its id
            except Exception as e:
                pass


In [ ]:
create_training_data()

In [8]:
print(len(training_data))

0


In [ ]:
print(len(training_data))
with open("training_data.txt", "wb") as fp:   #Pickling
  pickle.dump(training_data, fp)

10775


In [12]:
with open("drive//MyDrive//Colab Notebooks//training_data.txt", "rb") as fp:   # Unpickling
  training_data = pickle.load(fp)
print(len(training_data))


10775


In [13]:
import random
random.shuffle(training_data)
#shuffeling the data so that model will not learn the sequence

In [ ]:
#now split id and imageArray and save it in different list

In [14]:
X=[]
y=[]

for feat,lable in training_data:
    X.append(feat)
    y.append(lable)

##learn 
X= np.array(X).reshape(-1,imgSize,imgSize,3) # converting it to 4 dimension

In [ ]:
''' x = tf.convert_to_tensor(X)
y = tf.convert_to_tensor(Y)
x=X
y=Y '''

In [15]:
X.shape

(10775, 224, 224, 3)

In [16]:
#normalize the data
X=X/255.0;

In [17]:
Y=np.array(y) #covert list into np array
Y.shape

(10775,)

In [ ]:
#train deep learning model by transfer learning

In [1]:
#importing some other files
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
model= tf.keras.applications.MobileNetV2() # Pre trained model

14540800/14536120 [==============================] - 0s 0us/step


In [5]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
# transfer learning - tuning, weights will start from last

In [20]:
base_input = model.layers[0].input  #1st layer
base_output = model. layers[-2].output #last layer so that we can convert 1000 classess to 7 classess since our emotions are only 7

In [ ]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [21]:
final_output = layers.Dense(128)(base_output) # adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output) # we have 7 classes/emotions #classification layer

In [ ]:
final_output = layers.Dense(128)(base_output) # adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(4,activation='softmax')(final_output) # we have 7 classes/emotions #classification layer

In [ ]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [22]:
newmodel= keras.Model(inputs= base_input,outputs= final_output)

In [ ]:
newmodel.summary()

In [23]:
newmodel.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
#newmodel.compile(loss="sparse_categorical_cross",optimizer="adam", metrics=["accuracy"])

In [ ]:
newmodel.fit(X,Y, epochs=50)

Epoch 1/50
337/337 [==============================] - 1192s 4s/step - loss: 1.4582 - accuracy: 0.4421
Epoch 2/50
337/337 [==============================] - 1200s 4s/step - loss: 1.2321 - accuracy: 0.5383
Epoch 3/50
337/337 [==============================] - 1204s 4s/step - loss: 1.1292 - accuracy: 0.5709
Epoch 4/50
337/337 [==============================] - 1205s 4s/step - loss: 1.0390 - accuracy: 0.6111
Epoch 5/50
337/337 [==============================] - 1204s 4s/step - loss: 0.9758 - accuracy: 0.6391
Epoch 6/50
337/337 [==============================] - 1198s 4s/step - loss: 0.8989 - accuracy: 0.6624
Epoch 7/50
337/337 [==============================] - 1175s 3s/step - loss: 0.8417 - accuracy: 0.6897
Epoch 8/50
337/337 [==============================] - 1193s 4s/step - loss: 0.7706 - accuracy: 0.7153
Epoch 9/50
337/337 [==============================] - 1206s 4s/step - loss: 0.7099 - accuracy: 0.7388
Epoch 10/50
337/337 [==============================] - 1222s 4s/step - loss: 0.651

In [ ]:
newmodel.save('drive//MyDrive//Colab Notebooks//trained_model50.h5')